In [1]:
!pip install catboost 
!pip install radiant_mlhub
!pip install rasterio

In [2]:
# Required libraries
import json
import numpy as np
import os
import pandas as pd
import rasterio
import tarfile
import xarray as xr

from catboost import CatBoostClassifier
from datetime import datetime
from pathlib import Path
from radiant_mlhub import Collection

# Data Download

In [3]:
os.environ['MLHUB_API_KEY'] = 'dccd868e3faa15def7a3edc41f2c63fd69343d592411fd4e29da6ad053d93dd3'

collections = [
    'ref_south_africa_crops_competition_v1_train_labels',
    'ref_south_africa_crops_competition_v1_train_source_s1', 
    'ref_south_africa_crops_competition_v1_test_labels',
    'ref_south_africa_crops_competition_v1_test_source_s1'
]

def download(collection_id):
    print(f'Downloading {collection_id}...')
    collection = Collection.fetch(collection_id)
    path = collection.download('.')
    tar = tarfile.open(path, "r:gz")
    tar.extractall()
    tar.close()
    os.remove(path)
    
def resolve_path(base, path):
    return Path(os.path.join(base, path)).resolve()
    
def load_df(collection_id):
    collection = json.load(open(f'{collection_id}/collection.json', 'r'))
    rows = []
    item_links = []
    for link in collection['links']:
        if link['rel'] != 'item':
            continue
        item_links.append(link['href'])
        
    for item_link in item_links:
        item_path = f'{collection_id}/{item_link}'
        current_path = os.path.dirname(item_path)
        item = json.load(open(item_path, 'r'))
        tile_id = item['id'].split('_')[-1]
        for asset_key, asset in item['assets'].items():
            rows.append([
                tile_id,
                None,
                None,
                asset_key,
                str(resolve_path(current_path, asset['href']))
            ])
            
        for link in item['links']:
            if link['rel'] != 'source':
                continue
            link_path = resolve_path(current_path, link['href'])
            source_path = os.path.dirname(link_path)
            try:
                source_item = json.load(open(link_path, 'r'))
            except FileNotFoundError:
                continue
            datetime = source_item['properties']['datetime']
            satellite_platform = source_item['collection'].split('_')[-1]
            for asset_key, asset in source_item['assets'].items():
                rows.append([
                    tile_id,
                    datetime,
                    satellite_platform,
                    asset_key,
                    str(resolve_path(source_path, asset['href']))
                ])
    return pd.DataFrame(rows, columns=['tile_id', 'datetime', 'satellite_platform', 'asset', 'file_path'])

for c in collections:
    download(c)

train_df = load_df('ref_south_africa_crops_competition_v1_train_labels')
test_df = load_df('ref_south_africa_crops_competition_v1_test_labels')

# Data Processing

In [4]:
MODEL_DATES = [datetime(2017,4,15),
               datetime(2017,5,15),
               datetime(2017,6,15), 
               datetime(2017,7,15),
               datetime(2017,8,15),
               datetime(2017,9,15),
               datetime(2017,10,15)]
DATES = ["april",
         "may",
         "june", 
         "july",
         "august",
         "september",
         "october"] 
FEATURES = ["vv",
            "vh",
            "vv_spatial_mean",
            "vv_spatial_std",
            "vv_date_mean",
            "vv_date_std",
            "vh_spatial_mean",
            "vh_spatial_std",
            "vh_date_mean",
            "vh_date_std"]
COL_NAMES = [f"{feature}_{date}_mean" for feature in FEATURES for date in DATES]
COL_NAMES_STD = [f"{feature}_{date}_std" for feature in FEATURES for date in DATES]

In [5]:
def get_nearest_date(items, pivot):
    return min(items, key=lambda x: abs(datetime.strptime(x, '%Y-%m-%dT%H:%M:%S+0000') - pivot))

def extract_features(da, da_vars=["vv", "vh"]):
    for var in da_vars:
        da[f"{var}_spatial_mean"] = da[var].rolling(x=5, y=5, center=True, min_periods=1).mean()
        da[f"{var}_spatial_std"] = da[var].rolling(x=5, y=5, center=True, min_periods=1).std()
        da[f"{var}_date_mean"] = da[var].rolling(date=3, center=True, min_periods=1).mean()
        da[f"{var}_date_std"] = da[var].rolling(date=3, center=True, min_periods=1).std()    
    return da

def reshape_dataset_2dim(x):
    x = (x
         .to_array(dim="features")
         .transpose("y", "x", "date", "features")) 
    x = np.array(x).reshape(-1, x.shape[-1] * x.shape[-2]) 
    return x

In [6]:
def process_data(df, process_labels=True):
    tile_ids = df['tile_id'].unique()
    tile_list = list()
    i = 0
    for tile_id in tile_ids:
        if i%200 == 0:
            print(i)
        i += 1

        if tile_id != 1951: # avoid using this specific tile for the Hackathon as it might have a missing file
            tile_df = df[df['tile_id']==tile_id]
            if process_labels:
                # process labels
                tile_labels = xr.open_rasterio(tile_df[tile_df['asset']=='labels']['file_path'].values[0]).drop_vars(["band", "x", "y"])
            
            # process field IDs
            tile_field_ids = xr.open_rasterio(tile_df[tile_df['asset']=='field_ids']['file_path'].values[0]).drop_vars(["band", "x", "y"])
            
            # find tiles that match selected dates closest
            tile_date_times = tile_df[tile_df['satellite_platform']=='s1']['datetime'].unique()
            nearest_tile_date_times = [get_nearest_date(tile_date_times, date) for date in MODEL_DATES]
            
            # process Sentinel 1 vv and vh data for each timestep and concatenate 
            tile_timesteps = list()
            for date_time in nearest_tile_date_times:
                tile_timestep_vv = xr.open_rasterio(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='VV')]['file_path'].values[0]).drop_vars(["band", "x", "y"])
                tile_timestep_vh = xr.open_rasterio(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='VH')]['file_path'].values[0]).drop_vars(["band", "x", "y"])
                tile_timestep = xr.concat([tile_timestep_vv, tile_timestep_vh], dim="band")
                
                tile_timesteps.append(tile_timestep) 
            tile_features = xr.concat(tile_timesteps, dim="date")
            
        tile_features_ds = (tile_features
        .to_dataset(dim="band")
        .rename_vars({0: "vv", 1: "vh"}))
        tile_features_ds = extract_features(tile_features_ds)
        tile_features_np = reshape_dataset_2dim(tile_features_ds)
        
        if process_labels:
            labels_np = np.array(tile_labels.transpose("y", "x", "band")).reshape(-1,)
        field_ids_np = np.array(tile_field_ids.transpose("y", "x", "band")).reshape(-1,)    
        
        df_tile = pd.DataFrame(tile_features_np, columns=COL_NAMES)
        df_tile["field_id"] = field_ids_np
        if process_labels:
            df_tile["label"] = labels_np
        
        df_tile = df_tile[(df_tile.field_id != 0)]

        
        grouped_mean = df_tile.groupby('field_id').mean().reset_index().drop("field_id", axis=1)
        
        if process_labels:
            grouped_std = df_tile.drop("label", axis=1).groupby('field_id').std().reset_index()
        else:
            grouped_std = df_tile.groupby('field_id').std().reset_index()
        
        df_tile_grouped = pd.concat([grouped_std, grouped_mean], axis=1)
        df_tile_grouped["tile_id"] = tile_id

        tile_list.append(df_tile_grouped)
    
    return tile_list

In [37]:
train_tiles = process_data(train_df)
test_tiles = process_data(test_df, process_labels=False)

train_cols = ["field_id"] + COL_NAMES + COL_NAMES_STD + ["label", "tile_id"]
test_cols = ["field_id"] + COL_NAMES + COL_NAMES_STD + ["tile_id"]

train_data = pd.DataFrame(np.concatenate(train_tiles), columns=train_cols).dropna()
test_data = pd.DataFrame(np.concatenate(test_tiles), columns=test_cols)

# Model Fitting

In [8]:
np.random.seed(123)

In [ ]:
X = train_data.drop(columns=['label', 'tile_id', 'field_id'])
y = train_data['label'].astype(int)
cb = CatBoostClassifier(n_estimators=5000, depth=6, learning_rate=0.03, random_state=3)
cb.fit(X, y)

0:	learn: 2.1401112	total: 1.84s	remaining: 2h 33m 29s
1:	learn: 2.0917772	total: 2.8s	remaining: 1h 56m 49s
2:	learn: 2.0491892	total: 3.76s	remaining: 1h 44m 23s
3:	learn: 2.0123235	total: 4.7s	remaining: 1h 37m 45s
4:	learn: 1.9787026	total: 5.63s	remaining: 1h 33m 40s
5:	learn: 1.9500926	total: 6.52s	remaining: 1h 30m 25s
6:	learn: 1.9228137	total: 7.47s	remaining: 1h 28m 47s
7:	learn: 1.8970222	total: 8.37s	remaining: 1h 27m 5s
8:	learn: 1.8726966	total: 9.31s	remaining: 1h 26m 3s
9:	learn: 1.8511013	total: 10.2s	remaining: 1h 24m 28s
10:	learn: 1.8313738	total: 11.1s	remaining: 1h 23m 46s
11:	learn: 1.8124286	total: 12s	remaining: 1h 22m 48s
12:	learn: 1.7953201	total: 12.9s	remaining: 1h 22m 18s
13:	learn: 1.7780724	total: 13.9s	remaining: 1h 22m 14s
14:	learn: 1.7615678	total: 14.8s	remaining: 1h 21m 51s
15:	learn: 1.7466334	total: 15.6s	remaining: 1h 21m 6s
16:	learn: 1.7316527	total: 16.6s	remaining: 1h 20m 57s
17:	learn: 1.7177955	total: 17.5s	remaining: 1h 20m 44s
18:	learn

# Model Prediction
The submission evaluation did not work properly which is why two workarounds are used here. First a different column order to the tutorial notebook was used as the submission evaluation likely was using this order instead of the one in the tutorial notebook. Additionally, rows for tiles that weren't in the test data but were required for the submission upload were added. For those tiles each class was given equal probability of 1/9th.

In [43]:
# predict on submission data
test_prob = cb.predict_proba(test_data.drop(columns=['field_id', 'tile_id']))

In [44]:
# In this part we format the DataFrame to have column names and order similar to the sample submission file. 
pred_df = pd.DataFrame(test_prob)
pred_df = pred_df.rename(columns={
    7:'Crop_ID_1',
    2:'Crop_ID_2', 
    0:'Crop_ID_3',
    1:'Crop_ID_4',
    8:'Crop_ID_5',
    5:'Crop_ID_6',
    4:'Crop_ID_7',
    6:'Crop_ID_8',
    3:'Crop_ID_9'
})
pred_df['field_id'] = test_data['field_id']
pred_df = pred_df[['field_id', 'Crop_ID_1', 'Crop_ID_2', 'Crop_ID_3', 'Crop_ID_4', 'Crop_ID_5', 'Crop_ID_6', 'Crop_ID_7', 'Crop_ID_8', 'Crop_ID_9']]
pred_df

,field_id,Crop_ID_1,Crop_ID_2,Crop_ID_3,Crop_ID_4,Crop_ID_5,Crop_ID_6,Crop_ID_7,Crop_ID_8,Crop_ID_9
0,1825,0.019020,0.085486,0.122485,0.268347,0.006062,0.376861,0.056608,0.063459,0.001672
1,3079,0.022585,0.029792,0.179399,0.431321,0.008003,0.272181,0.036192,0.018579,0.001950
2,3848,0.014224,0.013161,0.113744,0.632718,0.002662,0.172232,0.017429,0.033648,0.000180
3,28857,0.001047,0.052636,0.036109,0.117160,0.009979,0.458202,0.044451,0.279293,0.001124
4,33478,0.015788,0.045261,0.186668,0.249270,0.002254,0.397939,0.028095,0.074293,0.000431
...,...,...,...,...,...,...,...,...,...,...
35290,95767,0.001592,0.024317,0.160713,0.640643,0.001738,0.108407,0.008911,0.015639,0.038042
35291,101421,0.001002,0.000256,0.000666,0.001522,0.000001,0.055051,0.000110,0.941220,0.000171
35292,105889,0.003748,0.028200,0.134613,0.294218,0.000614,0.043148,0.016702,0.004394,0.474364
35293,115157,0.007924,0.021847,0.240617,0.584447,0.000727,0.131414,0.005046,0.007561,0.000417


In [45]:
unavailable_ids = [
 1504,
 1544,
 2613,
 2747,
 5123,
 5813,
 5893,
 6051,
 9384,
 11510,
 13408,
 14534,
 14652,
 20502,
 20791,
 23273,
 23428,
 24440,
 24586,
 24844,
 27550,
 28366,
 29607,
 30101,
 31093,
 31121,
 34576,
 38466,
 39273,
 39903,
 42255,
 42862,
 43400,
 44399,
 46830,
 46831,
 48390,
 50379,
 50561,
 53686,
 55156,
 56280,
 56822,
 57008,
 59078,
 60581,
 63020,
 64002,
 64807,
 64989,
 65038,
 65804,
 67244,
 71551,
 72075,
 75407,
 75709,
 76235,
 77452,
 78113,
 79396,
 80161,
 80726,
 82251,
 82904,
 83109,
 84131,
 84518,
 85427,
 86358,
 86742,
 91681,
 92086,
 93155,
 93725,
 94169,
 94861,
 95252,
 96629,
 96746,
 99478,
 100120,
 102026,
 102162,
 103802,
 104605,
 107647,
 108168,
 109552,
 109825,
 111757,
 112178,
 113509,
 121814
 ]

In [46]:
for field_id in unavailable_ids:
    pred_df.loc[pred_df.shape[0]] = [field_id] + [0.12] + [0.11]*8

pred_df["field_id"] = pred_df["field_id"].astype("int")

In [47]:
pred_df.to_csv('final_submission.csv', index=False)